# Bedrock AgentCore Gateway を使用して OpenAPI を MCP ツールに変換する

## 概要
お客様は JSON または YAML 形式の OpenAPI スペックを持ち込み、Bedrock AgentCore Gateway を使用して API を MCP ツールに変換できます。ここでは、API キーを使用して NASA の Open API を呼び出す火星気象エージェントの構築を実演します。

Gateway のワークフローには、エージェントを外部ツールに接続するための以下のステップが含まれます：
* **Gateway 用のツールを作成** - REST API 用の OpenAPI 仕様などのスキーマを使用してツールを定義します。OpenAPI 仕様は Amazon Bedrock AgentCore によって解析され、Gateway が作成されます。
* **Gateway エンドポイントを作成** - インバウンド認証を備えた MCP エントリポイントとして機能するゲートウェイを作成します。
* **Gateway にターゲットを追加** - ゲートウェイがリクエストを特定のツールにルーティングする方法を定義する OpenAPI ターゲットを設定します。OpenAPI ファイルに含まれるすべての API が MCP 互換ツールとなり、Gateway エンドポイント URL を通じて利用可能になります。各 OpenAPI Gateway ターゲットのアウトバウンド認可を設定します。
* **エージェントコードを更新** - 統合された MCP インターフェースを通じてすべての設定済みツールにアクセスするため、エージェントを Gateway エンドポイントに接続します。

![仕組み](images/openapi-gateway-apikey.png)

### チュートリアル詳細

| 情報                  | 詳細                                                      |
|:---------------------|:----------------------------------------------------------|
| チュートリアルタイプ    | インタラクティブ                                           |
| AgentCore コンポーネント | AgentCore Gateway, AgentCore Identity                     |
| エージェントフレームワーク | Strands Agents                                            |
| Gateway ターゲットタイプ | OpenAPI                                                   |
| エージェント           | 火星気象エージェント                                       |
| インバウンド認証 IdP    | Amazon Cognito                                            |
| アウトバウンド認証      | API キー                                                   |
| LLM モデル             | Anthropic Claude Haiku 4.5, Amazon Nova Pro              |
| チュートリアル構成      | AgentCore Gateway の作成と呼び出し                         |
| チュートリアル業界      | クロスバーティカル                                         |
| 難易度                 | 簡単                                                       |
| 使用 SDK               | boto3                                                     |

チュートリアルの最初のパートでは、AgentCore Gateway ターゲットを作成します。

### チュートリアルアーキテクチャ
このチュートリアルでは、OpenAPI yaml/json ファイルで定義された操作を MCP ツールに変換し、Bedrock AgentCore Gateway でホストします。
デモンストレーション目的で、火星の気象に関するクエリに回答する火星気象エージェントを構築します。エージェントは NASA の Open API を使用します。ソリューションは Amazon Bedrock モデルを使用した Strands エージェントを使用します。
この例では、火星の気象用の getInsightWeather ツールを持つ非常にシンプルなエージェントを使用します。

## 前提条件

このチュートリアルを実行するには以下が必要です：
* Jupyter notebook（Python カーネル）
* uv
* AWS 認証情報
* Amazon Cognito

In [ ]:
!pip install --force-reinstall -U -r requirements.txt --quiet

In [ ]:
# Amazon SageMaker ノートブックを使用していない場合は AWS 認証情報を設定
import os
# os.environ['AWS_ACCESS_KEY_ID'] = '' # アクセスキーを設定
# os.environ['AWS_SECRET_ACCESS_KEY'] = '' # シークレットキーを設定
os.environ['AWS_DEFAULT_REGION'] = os.environ.get('AWS_REGION', 'us-east-1')

In [ ]:
import os
import sys

# 現在のスクリプトのディレクトリを取得
if '__file__' in globals():
    current_dir = os.path.dirname(os.path.abspath(__file__))
else:
    current_dir = os.getcwd()  # __file__ が定義されていない場合のフォールバック（例：Jupyter）

# utils.py を含むディレクトリに移動（1階層上）
utils_dir = os.path.abspath(os.path.join(current_dir, '../..'))

# sys.path に追加
sys.path.insert(0, utils_dir)

# これで utils をインポートできます
import utils

In [ ]:
#### Gateway が引き受ける IAM ロールを作成
import utils

agentcore_gateway_iam_role = utils.create_agentcore_gateway_role("sample-lambdagateway")
print("Agentcore Gateway ロール ARN: ", agentcore_gateway_iam_role['Role']['Arn'])

# Gateway へのインバウンド認可用に Amazon Cognito プールを作成

In [ ]:
# Cognito ユーザープールを作成
import os
import boto3
import requests
import time
from botocore.exceptions import ClientError

REGION = os.environ['AWS_DEFAULT_REGION']
USER_POOL_NAME = "sample-agentcore-gateway-pool"
RESOURCE_SERVER_ID = "sample-agentcore-gateway-id"
RESOURCE_SERVER_NAME = "sample-agentcore-gateway-name"
CLIENT_NAME = "sample-agentcore-gateway-client"
SCOPES = [
    {"ScopeName": "gateway:read", "ScopeDescription": "Read access"},
    {"ScopeName": "gateway:write", "ScopeDescription": "Write access"}
]
scopeString = f"{RESOURCE_SERVER_ID}/gateway:read {RESOURCE_SERVER_ID}/gateway:write"

cognito = boto3.client("cognito-idp", region_name=REGION)

print("Cognito リソースを作成または取得中...")
user_pool_id = utils.get_or_create_user_pool(cognito, USER_POOL_NAME)
print(f"ユーザープール ID: {user_pool_id}")

utils.get_or_create_resource_server(cognito, user_pool_id, RESOURCE_SERVER_ID, RESOURCE_SERVER_NAME, SCOPES)
print("リソースサーバーを確保しました。")

client_id, client_secret  = utils.get_or_create_m2m_client(cognito, user_pool_id, CLIENT_NAME, RESOURCE_SERVER_ID)
print(f"クライアント ID: {client_id}")

# ディスカバリー URL を取得
cognito_discovery_url = f'https://cognito-idp.{REGION}.amazonaws.com/{user_pool_id}/.well-known/openid-configuration'
print(cognito_discovery_url)

# Gateway を作成

In [ ]:
# CMK なしの Cognito 認可タイプで Gateway を作成。前のステップで作成した Cognito ユーザープールを使用
import boto3
gateway_client = boto3.client('bedrock-agentcore-control', region_name = os.environ['AWS_DEFAULT_REGION'])
auth_config = {
    "customJWTAuthorizer": { 
        "allowedClients": [client_id],  # クライアントは Cognito で設定された ClientId と一致する必要があります。例: 7rfbikfsm51j2fpaggacgng84g
        "discoveryUrl": cognito_discovery_url
    }
}
create_response = gateway_client.create_gateway(name='DemoGWOpenAPIAPIKeyNasaOAI',
    roleArn = agentcore_gateway_iam_role['Role']['Arn'], # IAM ロールには Gateway の作成/リスト/取得/削除の権限が必要です
    protocolType='MCP',
    authorizerType='CUSTOM_JWT',
    authorizerConfiguration=auth_config, 
    description='AgentCore Gateway with OpenAPI target'
)
print(create_response)
# GatewayTarget 作成に使用する GatewayID を取得
gatewayID = create_response["gatewayId"]
gatewayURL = create_response["gatewayUrl"]
print(gatewayID)

# Bedrock AgentCore Gateway を使用して NASA Open API を MCP ツールに変換

NASA の Open API から気象データを取得する火星気象エージェントを構築します。NASA Insight API に[こちら](https://api.nasa.gov/)で登録する必要があります。無料です！登録すると、メールで API キーが届きます。この API キーを使用して、OpenAPI ターゲット作成用の認証情報プロバイダーを設定します。

In [ ]:
import boto3
from pprint import pprint
from botocore.config import Config

acps = boto3.client(service_name="bedrock-agentcore-control")

response=acps.create_api_key_credential_provider(
    name="NasaInsightAPIKey",
    apiKey="eZgbDe0Q52J5Fd2HAnQ6FuXudghZVeVCGgMmXWWy", # Get an API key by signing up at api.nasa.gov. Takes 2-min to get an API key in your email.
)

pprint(response)
credentialProviderARN = response['credentialProviderArn']
pprint(f"Egress クレデンシャルプロバイダー ARN: {credentialProviderARN}")

# OpenAPI ターゲットを作成

#### NASA Open API json ファイルを S3 にアップロード

In [ ]:
# S3 クライアントを作成
session = boto3.session.Session()
s3_client = session.client('s3')
sts_client = session.client('sts')

# AWS アカウント ID とリージョンを取得
account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
# パラメータを定義
# OpenAPI json ファイルをアップロードする S3 バケット
bucket_name = f'agentcore-gateway-{account_id}-{region}'
file_path = 'openapi-specs/nasa_mars_insights_openapi.json'
object_key = 'nasa_mars_insights_openapi.json'
# put_object を使用してファイルをアップロードし、レスポンスを読み取る
try:
    if region == "us-east-1":
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name
        )
    else:
        s3bucket = s3_client.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': region
            }
        )
    with open(file_path, 'rb') as file_data:
        response = s3_client.put_object(
            Bucket=bucket_name,
            Key=object_key,
            Body=file_data
        )

    # アカウント ID とリージョンを含むアップロードされたオブジェクトの ARN を構築
    openapi_s3_uri = f's3://{bucket_name}/{object_key}'
    print(f'アップロードされたオブジェクトの S3 URI: {openapi_s3_uri}')
except Exception as e:
    print(f'ファイルアップロードエラー: {e}')

#### アウトバウンド認証を設定し、Gateway ターゲットを作成

In [ ]:
# OpenAPI 仕様ファイルの S3 URI
nasa_openapi_s3_target_config = {
    "mcp": {
          "openApiSchema": {
              "s3": {
                  "uri": openapi_s3_uri
              }
          }
      }
}

# API キー認証情報プロバイダーの設定
api_key_credential_config = [
    {
        "credentialProviderType" : "API_KEY", 
        "credentialProvider": {
            "apiKeyCredentialProvider": {
                    "credentialParameterName": "api_key", # 各 API プロバイダーが期待する API キー名に置き換えてください。ヘッダーでトークンを渡す場合は "Authorization" を使用
                    "providerArn": credentialProviderARN,
                    "credentialLocation":"QUERY_PARAMETER", # API キーの場所。可能な値は "HEADER" と "QUERY_PARAMETER"
                    #"credentialPrefix": " " # トークンのプレフィックス。有効な値は "Basic"。トークンにのみ適用されます
            }
        }
    }
  ]

targetname='DemoOpenAPITargetS3NasaMars'
response = gateway_client.create_gateway_target(
    gatewayIdentifier=gatewayID,
    name=targetname,
    description='OpenAPI Target with S3Uri using SDK',
    targetConfiguration=nasa_openapi_s3_target_config,
    credentialProviderConfigurations=api_key_credential_config)

# Strands エージェントから Bedrock AgentCore Gateway を呼び出す

Strands エージェントは、Model Context Protocol (MCP) 仕様を実装した Bedrock AgentCore Gateway を通じて AWS ツールとシームレスに統合します。この統合により、AI エージェントと AWS サービス間の安全で標準化された通信が可能になります。

Bedrock AgentCore Gateway は、基本的な MCP API である ListTools と InvokeTools を公開するプロトコル準拠のゲートウェイとして機能します。これらの API により、任意の MCP 準拠クライアントまたは SDK が利用可能なツールを発見し、安全で標準化された方法で対話できます。Strands エージェントが AWS サービスにアクセスする必要がある場合、これらの MCP 標準化されたエンドポイントを使用して Gateway と通信します。

Gateway の実装は [MCP 認可仕様](https://modelcontextprotocol.org/specification/draft/basic/authorization) に厳密に準拠しており、堅牢なセキュリティとアクセス制御を保証します。これは、Strands エージェントによるすべてのツール呼び出しが認可ステップを経ることを意味し、強力な機能を有効にしながらセキュリティを維持します。

例えば、Strands エージェントが MCP ツールにアクセスする必要がある場合、まず ListTools を呼び出して利用可能なツールを発見し、次に InvokeTools を使用して特定のアクションを実行します。Gateway はすべての必要なセキュリティ検証、プロトコル変換、サービス対話を処理し、プロセス全体をシームレスかつ安全にします。

このアーキテクチャアプローチにより、MCP 仕様を実装する任意のクライアントまたは SDK が Gateway を通じて AWS サービスと対話できるため、AI エージェント統合のための汎用性が高く将来を見据えたソリューションとなります。

# インバウンド認可用に Amazon Cognito からアクセストークンをリクエスト

In [ ]:
print("Amazon Cognito 認可タイプからアクセストークンをリクエスト中...ドメイン名の伝播が完了するまで失敗する場合があります")
token_response = utils.get_token(user_pool_id, client_id, client_secret,scopeString,REGION)
token = token_response["access_token"]
print("トークンレスポンス:", token)

# Bedrock AgentCore Gateway を使用して NASA Open API を呼び出し、火星気象エージェントに質問

In [ ]:
from strands.models import BedrockModel
from mcp.client.streamable_http import streamablehttp_client 
from strands.tools.mcp.mcp_client import MCPClient
from strands import Agent

def create_streamable_http_transport():
    return streamablehttp_client(gatewayURL,headers={"Authorization": f"Bearer {token}"})

client = MCPClient(create_streamable_http_transport)

## ~/.aws/credentials で設定された IAM グループ/ユーザーは Bedrock モデルへのアクセス権が必要です
yourmodel = BedrockModel(
    model_id="us.amazon.nova-pro-v1:0",
    temperature=0.7,
)

In [ ]:
from strands import Agent
import logging


# strands のルートロガーを設定。問題をデバッグする場合は DEBUG に変更してください
logging.getLogger("strands").setLevel(logging.INFO)

# ログを表示するためのハンドラーを追加
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

with client:
    # listTools を呼び出す
    tools = client.list_tools_sync()
    # モデルとツールを持つエージェントを作成
    agent = Agent(model=yourmodel,tools=tools) ## お好みのモデルに置き換え可能
    print(f"エージェントに読み込まれたツール: {agent.tool_names}")
    #print(f"エージェントのツール設定: {agent.tool_config}")
    # サンプルプロンプトでエージェントを呼び出す。これは MCP listTools のみを呼び出し、LLM がアクセスできるツールのリストを取得します。以下は実際にはツールを呼び出しません
    agent("利用可能なすべてのツールをリストしてもらえますか")
    agent("火星の北部の天気はどうですか")
    # サンプルプロンプトでエージェントを呼び出し、ツールを呼び出してレスポンスを表示
    # MCP ツールを明示的に呼び出す。MCP ツール名と引数は AWS Lambda 関数または OpenAPI/Smithy API と一致する必要があります
    result = client.call_tool_sync(
    tool_use_id="get-insight-weather-1", # 一意の識別子に置き換え可能
    name=targetname+"___getInsightWeather", # これは AWS Lambda ターゲットタイプに基づくツール名。ターゲット名によって変わります
    arguments={"ver": "1.0","feedtype": "json"}
    )
    # MCP ツールのレスポンスを表示
    print(f"ツール呼び出し結果: {result['content'][0]['text']}")

**問題: 以下のセルを実行時に下記のエラーが発生する場合、pydantic と pydantic-core バージョン間の非互換性を示しています。**

```
TypeError: model_schema() got an unexpected keyword argument 'generic_origin'
```
**解決方法**

互換性のある pydantic==2.7.2 と pydantic-core 2.27.2 がインストールされていることを確認してください。完了後、カーネルを再起動してください。

# クリーンアップ
IAM ロール、IAM ポリシー、認証情報プロバイダー、AWS Lambda 関数、Cognito ユーザープール、S3 バケットなどの追加リソースも作成されます。クリーンアップの一環として手動で削除する必要がある場合があります。これは実行する例によって異なります。

## Gateway を削除（オプション）

In [ ]:
import utils
utils.delete_gateway(gateway_client,gatewayID)